In [3]:
import numpy as np
import pandas as pd
from matplotlib .pyplot import subplots
import statsmodels .api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
(LinearDiscriminantAnalysis as LDA ,
QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [4]:
Smarket = load_data ('Smarket')
Smarket

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
...,...,...,...,...,...,...,...,...,...
1245,2005,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
1246,2005,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
1247,2005,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up
1248,2005,0.130,-0.955,0.043,0.422,0.252,1.42236,-0.298,Down


In [5]:
allvars = Smarket.columns.drop (['Today', 'Direction', 'Year'])
design = MS(allvars)
X = design.fit_transform (Smarket)
y = Smarket.Direction == 'Up'

In [6]:
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
X_train, X_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]
D = Smarket.Direction
L_train, L_test = D.loc[train], D.loc[~train]

In [7]:
X_train, X_test = [M.drop(columns=['intercept'])
 for M in [X_train, X_test]]

In [8]:
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, L_train)
knn1_pred = knn1.predict(X_test)
confusion_table(knn1_pred, L_test)

Truth,Down,Up
Predicted,,
Down,50,62
Up,61,79


In [9]:
(83+43)/252, np.mean(knn1_pred == L_test)

(0.5, np.float64(0.5119047619047619))

In [11]:
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3_pred = knn3.fit(X_train, L_train).predict(X_test)
np.mean(knn3_pred == L_test)

np.float64(0.503968253968254)

In [12]:
Caravan = load_data('Caravan')
Purchase = Caravan.Purchase
Purchase.value_counts()

Purchase
No     5474
Yes     348
Name: count, dtype: int64

In [13]:
feature_df = Caravan.drop(columns=['Purchase'])

In [14]:
scaler = StandardScaler(with_mean=True,
 with_std=True,
 copy=True)

In [15]:
scaler.fit(feature_df)
X_std = scaler.transform(feature_df)

In [17]:
feature_std = pd.DataFrame(
    X_std,
    columns=feature_df.columns);
feature_std.std()

MOSTYPE     1.000086
MAANTHUI    1.000086
MGEMOMV     1.000086
MGEMLEEF    1.000086
MOSHOOFD    1.000086
              ...   
AZEILPL     1.000086
APLEZIER    1.000086
AFIETS      1.000086
AINBOED     1.000086
ABYSTAND    1.000086
Length: 85, dtype: float64

In [18]:
(X_train,
 X_test,
 y_train,
 y_test) = train_test_split(feature_std,
 Purchase,
 test_size=1000,
 random_state=0)

In [19]:
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1_pred = knn1.fit(X_train, y_train).predict(X_test)
np.mean(y_test != knn1_pred), np.mean(y_test != "No")

(np.float64(0.111), np.float64(0.067))

In [20]:
confusion_table(knn1_pred, y_test)

Truth,No,Yes
Predicted,,
No,880,58
Yes,53,9


In [22]:
9/(53+9)

0.14516129032258066

In [24]:
for K in range (1,6) :
 knn = KNeighborsClassifier(n_neighbors=K)
 knn_pred = knn.fit(X_train, y_train).predict(X_test)
 C = confusion_table(knn_pred, y_test)
 templ = ('K={0:d}: # predicted to rent: {1:>2},' +
          '# who did rent {2:d}, accuracy {3:.1%}')
 pred = C.loc['Yes'].sum()
 did_rent = C.loc['Yes','Yes']
 print(templ.format(
 K,
 pred,
 did_rent,
 did_rent / pred))

K=1: # predicted to rent: 62,# who did rent 9, accuracy 14.5%
K=2: # predicted to rent:  6,# who did rent 1, accuracy 16.7%
K=3: # predicted to rent: 20,# who did rent 3, accuracy 15.0%
K=4: # predicted to rent:  4,# who did rent 0, accuracy 0.0%
K=5: # predicted to rent:  7,# who did rent 1, accuracy 14.3%
